# 轴转置基准测试
本笔记本比较了不同轴转置实现的性能。

**注意：** 基准测试结果会根据图像大小、内核大小、使用的操作、参数和使用的硬件而有很大变化。使用此笔记本来适应您的用例场景，并在您的目标硬件上进行基准测试。如果您有不同的场景或用例，非常欢迎您以拉取请求的形式提交您的笔记本！

In [1]:
import pyclesperanto_prototype as cle
import numpy as np
import time
import cupy as cp

# to measure kernel execution duration properly, we need to set this flag. It will slow down exection of workflows a bit though
cle.set_wait_for_kernel_finish(True)

# selet a GPU with the following in the name. This will fallback to any other GPU if none with this name is found
cle.select_device('RTX')

<NVIDIA GeForce RTX 3050 Ti Laptop GPU on Platform: NVIDIA CUDA (1 refs)>

In [2]:
# test data
import numpy as np

test_image = np.random.random([100, 512, 1024])

## clEsperanto

In [3]:
# transpose with pyclesperanto
result_image = None

test_image_gpu = cle.push_zyx(test_image)

for i in range(0, 10):
    start_time = time.time()
    result_image = cle.transpose_xz(test_image_gpu, result_image)
    print("pyclesperanto transpose duration: " + str(time.time() - start_time))
print(result_image.shape)

pyclesperanto transpose duration: 0.06201291084289551
pyclesperanto transpose duration: 0.04100918769836426
pyclesperanto transpose duration: 0.040008544921875
pyclesperanto transpose duration: 0.040008544921875
pyclesperanto transpose duration: 0.0400090217590332
pyclesperanto transpose duration: 0.04129624366760254
pyclesperanto transpose duration: 0.042009592056274414
pyclesperanto transpose duration: 0.04128861427307129
pyclesperanto transpose duration: 0.04102063179016113
pyclesperanto transpose duration: 0.04099869728088379
(1024, 512, 100)


## cupy

In [4]:
# transpose with numpy
result_image = None
cu_test_image = cp.asarray(test_image)

for i in range(0, 10):
    start_time = time.time()
    result_image = cp.transpose(cu_test_image, (2, 1, 0))
    cp.cuda.stream.get_current_stream().synchronize() # we need to wait here to measure time properly
    print("cupy transpose duration: " + str(time.time() - start_time))
print(result_image.shape)

cupy transpose duration: 0.06301379203796387
cupy transpose duration: 0.0009999275207519531
cupy transpose duration: 0.0
cupy transpose duration: 0.0
cupy transpose duration: 0.0
cupy transpose duration: 0.0
cupy transpose duration: 0.0
cupy transpose duration: 0.0
cupy transpose duration: 0.0
cupy transpose duration: 0.0
(1024, 512, 100)


## numpy

In [5]:
# transpose with numpy
result_image = None

for i in range(0, 10):
    start_time = time.time()
    result_image = np.transpose(test_image, (2, 1, 0))
    print("numpy transpose duration: " + str(time.time() - start_time))
print(result_image.shape)

numpy transpose duration: 0.0
numpy transpose duration: 0.0
numpy transpose duration: 0.0
numpy transpose duration: 0.0
numpy transpose duration: 0.0
numpy transpose duration: 0.0
numpy transpose duration: 0.0
numpy transpose duration: 0.0
numpy transpose duration: 0.0
numpy transpose duration: 0.0
(1024, 512, 100)
